index


gender: 성별


car: 차량 소유 여부


reality: 부동산 소유 여부


child_num: 자녀 수


income_total: 연간 소득


income_type: 소득 분류


['Commercial associate', 'Working', 'State servant', 'Pensioner', 'Student']



edu_type: 교육 수준


['Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Academic degree']



family_type: 결혼 여부


['Married', 'Civil marriage', 'Separated', 'Single / not married', 'Widow']



house_type: 생활 방식


['Municipal apartment', 'House / apartment', 'With parents',

'Co-op apartment', 'Rented apartment', 'Office apartment']



DAYS_BIRTH: 출생일


데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전에 태어났음을 의미



DAYS_EMPLOYED: 업무 시작일(365243)


데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전부터 일을 시작함을 의미

양수 값은 고용되지 않은 상태를 의미함



FLAG_MOBIL: 핸드폰 소유 여부


work_phone: 업무용 전화 소유 여부


phone: 전화 소유 여부


email: 이메일 소유 여부


occyp_type: 직업 유형


family_size: 가족 규모


begin_month: 신용카드 발급 월
데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 한 달 전에 신용카드를 발급함을 의미

credit: 사용자의 신용카드 대금 연체를 기준으로 한 신용도


=> 낮을 수록 높은 신용의 신용카드 사용자를 의미함

개선사항  
======
1.DAYS_EMPLOYED에 기반한 일부 데이터 전처리하기  
2.여러 카드를 발급 받은 사람에 관한 데이터 열 추가하기

In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.preprocessing import MinMaxScaler
import random
import glob

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('submission.csv')

In [3]:
train.isnull().sum() #occyp_type에 NULL 데이터 8171개 존재 직업이 없는 사람 or 직업 미게재된 경우라 생각

index               0
gender              0
car                 0
reality             0
child_num           0
income_total        0
income_type         0
edu_type            0
family_type         0
house_type          0
DAYS_BIRTH          0
DAYS_EMPLOYED       0
FLAG_MOBIL          0
work_phone          0
phone               0
email               0
occyp_type       8171
family_size         0
begin_month         0
credit              0
dtype: int64

In [14]:
train.groupby(['occyp_type']).sum()


,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit
occyp_type,,,,,,,,,,,,
Accountants,11983649,411,184596052.5,-13407738,-2381162,902,262,314,124,1991.0,-24285.0,1341.0
Cleaning staff,5429812,156,59728500.0,-7073736,-972731,403,140,108,17,807.0,-9949.0,633.0
Cooking staff,6053572,170,66847950.0,-6836888,-1100831,457,91,127,35,979.0,-11484.0,688.0
Core staff,35313713,1574,506639223.0,-37493780,-7672885,2646,758,794,378,6237.0,-69359.0,3976.0
Drivers,20823004,843,332094600.0,-23499285,-3173613,1575,439,459,136,3757.0,-41662.0,2418.0
HR staff,784019,22,11556000.0,-881749,-115096,62,7,11,9,118.0,-1564.0,106.0
High skill tech staff,13963832,589,204466500.0,-15525409,-3228667,1040,296,372,135,2455.0,-28611.0,1564.0
IT staff,488211,16,8067600.0,-526418,-67200,41,5,9,14,88.0,-1070.0,56.0
Laborers,58522191,2210,813546040.5,-65910376,-12199236,4512,1186,1187,346,10242.0,-115297.0,6770.0


In [4]:
train = train.fillna('Unemployed') #일하고 있지 않은 상태로 결측치 채우기
test = test.fillna('Unemployed')

In [5]:
train = train.drop(['index'], axis=1)
test = test.drop(['index'], axis=1)

In [6]:
train.columns

Index(['gender', 'car', 'reality', 'child_num', 'income_total', 'income_type',
       'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'occyp_type',
       'family_size', 'begin_month', 'credit'],
      dtype='object')

1. 두가지 레이블이 있는 경우: 
 - obeject : gender,car,reality
 - int : work_phone, phone, email

2. 여러가지 레이블 but 숫자  
child_num : 0,1,2,3,4,5,14,19,7 => 14 와 19의 경우 너무 높은 값이므로 개별 데이터 확인 필요  
income_total: min(27000) max(1575000) max 값이 min 값의 58배 이상  
DAYS_BIRTH: min(-25152), max(-7705) 음수 값에 대한 데이터 처리 고려 해야함  
DAYS_EMPLOYED: min(-15713), max(365243) 음수 값에 대한 데이터 처리 고려 해야함  
family_size: 2,3,4,1,5,6,7,15,20,9 => child_num과 같이 고려해야할 데이터   
begin_month : min(-60), max(0) 음수 값에 대한 데이터 처리 고려 해야함
credit: 1,2,0

3. 레이블이 두개보다 많은 경우: 
family_type 5가지  
intcome_type 5가지  
edu_type 5가지  
house_type 6가지  
occyp_type 19가지  

4. 그외
FLAG_MOBIL 1 하나만 존재

In [7]:
train['credit'].unique()

array([1., 2., 0.])

In [8]:
train.dtypes

gender            object
car               object
reality           object
child_num          int64
income_total     float64
income_type       object
edu_type          object
family_type       object
house_type        object
DAYS_BIRTH         int64
DAYS_EMPLOYED      int64
FLAG_MOBIL         int64
work_phone         int64
phone              int64
email              int64
occyp_type        object
family_size      float64
begin_month      float64
credit           float64
dtype: object

In [9]:
LE_col = ['gender', 'car', 'reality']
OH_col = ['income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']

In [10]:
classle = LabelEncoder()
LE_train=pd.DataFrame()
classle.fit(train.loc[:,LE_col[0]])
LE_train[LE_col[0]] =classle.transform(train.loc[:,LE_col[0]])
test[LE_col[0]] =classle.transform(test.loc[:,LE_col[0]])
classle.fit(train.loc[:,LE_col[1]])
LE_train[LE_col[1]] =classle.transform(train.loc[:,LE_col[1]])
test[LE_col[1]] =classle.transform(test.loc[:,LE_col[1]])
classle.fit(train.loc[:,LE_col[2]])
LE_train[LE_col[2]] =classle.transform(train.loc[:,LE_col[2]])
test[LE_col[2]] =classle.transform(test.loc[:,LE_col[2]])

In [11]:
LE_train

,gender,car,reality
0,0,0,0
1,0,0,1
2,1,1,1
3,0,0,1
4,0,1,1
...,...,...,...
26452,0,0,0
26453,0,0,1
26454,0,1,0
26455,1,0,1


In [12]:
ohe = OneHotEncoder()
ohe.fit(train.loc[:,OH_col])
OH_train=pd.DataFrame()
OH_train = pd.DataFrame(ohe.transform(train.loc[:,OH_col]).toarray(), columns=ohe.get_feature_names(OH_col))
OH_test=pd.DataFrame()
OH_test = pd.DataFrame(ohe.transform(test.loc[:,OH_col]).toarray(), columns=ohe.get_feature_names(OH_col))

In [13]:
train_x = train.drop(LE_col, axis=1)
train_x= train_x.drop(OH_col, axis=1)
train_x =pd.concat([train_x, LE_train], axis=1)
train_x =pd.concat([train_x, OH_train], axis=1)
test = test.drop(OH_col, axis=1)
test = pd.concat([test, OH_test], axis=1)

In [14]:
train_x['gender']

0        0
1        0
2        1
3        0
4        0
        ..
26452    0
26453    0
26454    0
26455    1
26456    0
Name: gender, Length: 26457, dtype: int32

In [15]:
train_x['DAYS_BIRTH']= train_x['DAYS_BIRTH']*(-1/365)
test['DAYS_BIRTH']= test['DAYS_BIRTH']*(-1/365)

In [16]:
train_x['DAYS_EMPLOYED'].loc[train_x['DAYS_EMPLOYED']>0]=0
train_x['DAYS_EMPLOYED'].loc[train_x['DAYS_EMPLOYED']<0]= train_x['DAYS_EMPLOYED'].loc[train_x['DAYS_EMPLOYED']<0]*(-1/365)
test['DAYS_EMPLOYED'].loc[test['DAYS_EMPLOYED']>0]=0
test['DAYS_EMPLOYED'].loc[test['DAYS_EMPLOYED']<0]= test['DAYS_EMPLOYED'].loc[test['DAYS_EMPLOYED']<0]*(-1/365)

C:\Users\yoons\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [17]:
train_x['begin_month']= train_x['begin_month']*(-1/12)
test['begin_month']= test['begin_month']*(-1/12)

sc=MinMaxScaler()
train_x['income_total']=sc.fit_transform(train_x.loc[:,train_x.columns=='income_total'])
test['income_total']=sc.transform(test.loc[:,test.columns=='income_total'])

In [25]:
train_x.columns

Index(['child_num', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'family_size',
       'begin_month', 'credit', 'gender', 'car', 'reality',
       'income_type_Commercial associate', 'income_type_Pensioner',
       'income_type_State servant', 'income_type_Student',
       'income_type_Working', 'edu_type_Academic degree',
       'edu_type_Higher education', 'edu_type_Incomplete higher',
       'edu_type_Lower secondary', 'edu_type_Secondary / secondary special',
       'family_type_Civil marriage', 'family_type_Married',
       'family_type_Separated', 'family_type_Single / not married',
       'family_type_Widow', 'house_type_Co-op apartment',
       'house_type_House / apartment', 'house_type_Municipal apartment',
       'house_type_Office apartment', 'house_type_Rented apartment',
       'house_type_With parents', 'occyp_type_Accountants',
       'occyp_type_Cleaning staff', 'occyp_type_Cooking staff',
       'occyp_type_Core s

In [24]:
test.columns

Index(['gender', 'car', 'reality', 'child_num', 'income_total', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'work_phone', 'phone', 'email',
       'family_size', 'begin_month', 'income_type_Commercial associate',
       'income_type_Pensioner', 'income_type_State servant',
       'income_type_Student', 'income_type_Working',
       'edu_type_Academic degree', 'edu_type_Higher education',
       'edu_type_Incomplete higher', 'edu_type_Lower secondary',
       'edu_type_Secondary / secondary special', 'family_type_Civil marriage',
       'family_type_Married', 'family_type_Separated',
       'family_type_Single / not married', 'family_type_Widow',
       'house_type_Co-op apartment', 'house_type_House / apartment',
       'house_type_Municipal apartment', 'house_type_Office apartment',
       'house_type_Rented apartment', 'house_type_With parents',
       'occyp_type_Accountants', 'occyp_type_Cleaning staff',
       'occyp_type_Cooking staff', 'occyp_type_Core staff',
       'oc

In [20]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train_x, train['credit']):
    folds.append((train_idx, valid_idx))

xgb_models={}
for fold in range(5):
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train_x.iloc[train_idx].values, train_x.iloc[valid_idx].values,train_y.iloc[train_idx].values, train_y.iloc[valid_idx].values 
    model = XGBClassifier(n_estimators=3000, learning_rate= 0.05, max_depth = 6)
    model.fit(X_train, y_train, eval_set =[(X_train, y_train), (X_valid, y_valid)], early_stopping_rounds=30, verbose=100)
    xgb_models[fold]=model

In [21]:
random.seed(42)
lgb_models={}
for fold in range(5):
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train_x.drop(['credit'],axis=1).iloc[train_idx].values, train_x.drop(['credit'],axis=1).iloc[valid_idx].values,train_x['credit'][train_idx].values, train_x['credit'][valid_idx].values 
    lgb = LGBMClassifier(n_estimators=1000)
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb

Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.65115	valid_1's multi_logloss: 0.750211
[200]	training's multi_logloss: 0.565182	valid_1's multi_logloss: 0.731945
[300]	training's multi_logloss: 0.504345	valid_1's multi_logloss: 0.729437
Early stopping, best iteration is:
[278]	training's multi_logloss: 0.516279	valid_1's multi_logloss: 0.728845
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.646185	valid_1's multi_logloss: 0.762279
[200]	training's multi_logloss: 0.55956	valid_1's multi_logloss: 0.750029
Early stopping, best iteration is:
[227]	training's multi_logloss: 0.54101	valid_1's multi_logloss: 0.74773
Training until validation scores don't improve for 30 rounds
[100]	training's multi_logloss: 0.651901	valid_1's multi_logloss: 0.759164
[200]	training's multi_logloss: 0.565156	valid_1's multi_logloss: 0.746958
Early stopping, best iteration is:
[242]	training's multi_logloss: 0.535393	

In [22]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(test)/5

In [23]:
submit.to_csv('submission.csv', index=False)

모델 제출 결과: XGBClassifier 2000/0.1 **3.47525**  
모델 제출 결과2: XGBClassifier 2000/0.01 **3.34525**  
모델 제출 결과3: XGBClassifier 2000/0.05 depth 6 **3.22263**
모델 제출 결과4: LGBMClassifier 1000 **3.01567**  